In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_after_attempt, wait_exponential
import time

In [ ]:
urls_df = pd.read_csv('cleaned_Case_links.csv')
urls_df['Links'] = urls_df['Links'].str.strip()  

In [ ]:
empty_urls = urls_df[urls_df['Links'].isna() | (urls_df['Links'] == '')]
if not empty_urls.empty:
    print("Empty URLs found:")
    print(empty_urls)

urls = urls_df['Links'].dropna().tolist()

In [ ]:
data = []
failed_urls = [] 

@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=2, min=5, max=20), reraise=True)
def fetch_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers, timeout=30)  
    response.raise_for_status()  
    return response

def scrape_url(url):
    if url == '':
        print("Empty URL found")
        return [None, None, None, None, url]

    try:
        time.sleep(1)

        page = fetch_page(url) 
        soup = BeautifulSoup(page.text, 'html.parser')

        span_elements = soup.find_all('span')
        h5_elements = soup.find_all('h5')
        paragraphs = soup.find('div', id='collapsefour')

        Number = None
        Date = None
        City = None
        Fact = None

        for span in span_elements:
            text = span.get_text().strip()
            if Number is None and (text.startswith('رقم القضية - القرار:') or text.startswith('رقم القرار:')):
                Number = text.replace('رقم القضية - القرار:', '').replace('رقم القرار:', '').strip()
            elif Date is None and (text.startswith('تاريخها:') or text.startswith('تاريخه:')):
                Date = text.replace('تاريخها:', '').replace('تاريخه:', '').strip()
            if Number and Date:
                break


        for h5 in h5_elements:
            text = h5.get_text().strip()
            if City is None and (text.startswith('المدينه:') or text.startswith('المدينة:')):
                City = text.replace('المدينه:', '').replace('المدينة:', '').strip()
                break

        # Extract Fact
        if paragraphs:
            Fact = paragraphs.get_text().strip()
        else:
            fact_element_alt = soup.find('div', id='collapsefive')
            if fact_element_alt:
                Fact = fact_element_alt.get_text().strip()

        return [Number, Date, City, Fact, url]

    except requests.exceptions.RequestException as e:
        print(f"Network error processing {url}: {e}")
        failed_urls.append(url)  # Log failed URL
        return [None, None, None, None, url]
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return [None, None, None, None, url]


with ThreadPoolExecutor(max_workers=5) as executor:  
    results = list(executor.map(scrape_url, urls))

data.extend(results)

df = pd.DataFrame(data, columns=['Number', 'Date', 'City', 'Fact', 'URL'])
df.to_csv('Case_Data.csv', index=False)
print("Data saved successfully to 'Case_Data.csv'.")

if failed_urls:
    pd.DataFrame(failed_urls, columns=["URL"]).to_csv("Failed_URLs.csv", index=False)
    print("Failed URLs saved to 'Failed_URLs.csv'.")

Data saved successfully to 'Case_Data.csv'.


In [4]:
df = pd.read_csv("Case_Data.csv" , engine='python')
df

,Number,Date,City,Fact,URL
0,٤٧١/٣/ق/,١١ / ٤ / ١٤٣٨,الدمام,الحكم الغيابي الصادر من الدائرة التجارية الراب...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13340
1,١٦٨٦/٣/ق/,١ / ١١ / ١٤٣٨,الدمام,الحكم في القضية رقم (١٦٨٦/٣/ق/ لعام ١٤٣٨ هـ)ال...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13341
2,٤٠١٢/٣/ق/,٢٥ / ١٠ / ١٤٣٨,الدمام,الحكم في القضية رقم ٤٠١٢/٣/ق/ لعام ١٤٣٢ هـالمق...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13343
3,٧١٥/٢/ق,٢١ / ٨ / ١٤٣٨,جدة,الحكم في القضية رقم ٧١٥/٢/ق لعام ١٤٣٨ هـالمقام...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13424
4,١٢٣٩١/٢/ق,١٧ / ٥ / ١٤٣٨,جدة,الحكم في القضية رقم ١٢٣٩١/٢/ق لعام ١٤٣٧هـالمقا...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13518
...,...,...,...,...,...
21607,٩٨٥٢/٣ /ق,١٢ / ٢ / ١٤٣٩,الدمام,الحكم الصادر في القضية التجارية رقم ٩٨٥٢/٣ /ق...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13601
21608,١٥٤٥/٢/ق,٢٥ / ٢ / ١٤٣٩,مكة المكرمة,الحكم الصادر في القضية رقم ١٥٤٥/٢/ق لعام ١٤٣٨ه...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13142
21609,٩٩٥٣/٣/ق,١ / ٤ / ١٤٣٩,الدمام,الحكم في القضية رقم (٩٩٥٣/٣/ق لعام ١٤٣٨هـ)المق...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13144
21610,١٠٤٦٥/٣/ق,١٧ / ٣ / ١٤٣٩,الدمام,الحكم في القضية رقم (١٠٤٦٥/٣/ق لعام ١٤٣٨هـ)الم...,https://sjp.moj.gov.sa/Filter/AhkamDetails/13162
